In [4]:
pip install gradio jupyter ipywidgets ollama

Note: you may need to restart the kernel to use updated packages.


In [5]:
import ollama
ollama.pull('mistral')

{'status': 'success'}

In [6]:
from ollama import Client
client = Client(host='http://localhost:11434')
response = client.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

 The color of the sky appears blue due to a phenomenon called Rayleigh scattering. When the sun's rays enter Earth's atmosphere, they collide with molecules and particles in the air, such as nitrogen and oxygen. Blue light has shorter wavelengths and gets scattered more easily than other colors, causing the sky to appear blue during the day. This effect is most noticeable when the sun is relatively low on the horizon, which is why we often see beautiful blue skies at dawn and dusk.


In [ ]:
import ollama

stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

In [ ]:
ollama.list()

In [ ]:
ollama.show('mistral:7b')

In [7]:
import gradio as gr
from ollama import Client

def mistralChat(text):
    
    client = Client(host='http://localhost:11434')
    response = client.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': text,
    },
    ])
    return response['message']['content']

In [8]:
with gr.Blocks() as server:
    with gr.Tab("LLM Inferencing"):
    
        model_input = gr.Textbox(label="Your Question:", 
                                value="What’s your question?", interactive=True)
        ask_button = gr.Button("Ask")
        model_output = gr.Textbox(label="The Answer:", interactive=False, 
                                value="Answer goes here...")
    
    ask_button.click(mistralChat, inputs=[model_input], outputs=[model_output])

server.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://fad33f6c11c6c44eae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
